In [51]:
import neo4j
from neo4j import GraphDatabase
import json
import json5
import random

In [52]:
def exec_query(
  uri: str,
  username: str,
  password: str,
  query: str, 
  params: dict,
  database: str = "neo4j"):
    with GraphDatabase.driver(uri, auth=(username, password)) as driver:
      records, summary, keys = driver.execute_query(query, params, database=database)
      return records, summary, keys



# RESET DB

In [53]:
delete_query = """
MATCH (n)
DETACH DELETE n
"""
exec_query("neo4j://localhost:7687", "neo4j", "password", delete_query, {})

([], <neo4j._work.summary.ResultSummary at 0x7b01a3b7b050>, [])

# DUMP ALL POKEMONS IN DB

In [ ]:
def createPokemon(pokemon):
    return {
        "name": pokemon["name"],
        "num": pokemon["num"],
        "heightm": pokemon["heightm"],
        "weightkg": pokemon["weightkg"],
        "hp": pokemon["baseStats"]["hp"],
        "atk": pokemon["baseStats"]["atk"],
        "def": pokemon["baseStats"]["def"],
        "spa": pokemon["baseStats"]["spa"],
        "spd": pokemon["baseStats"]["spd"],
        "spe": pokemon["baseStats"]["spe"]
    } 

def createMove(move):
    return {
        "name": move["name"],
        "accuracy": move["accuracy"],
        "basePower": move["basePower"],
        "pp": move["pp"],
        "category": move["category"],
        "description": move["shortDesc"]
    }


INSERT_POKEMON = """
CREATE (p:Pokemon 
    {
        name: $name, 
        num: $num, 
        heightm: $heightm, 
        weightkg: $weightkg,
        hp: $hp,
        atk: $atk,
        def: $def,
        spa: $spa,
        spd: $spd,
        spe: $spe
    }
)
"""

INSERT_ABILITY = """
CREATE (a:Ability {name: $name, description: $description})
"""

INSERT_TYPE = """
CREATE (t:Type {name: $name})
"""

INSERT_POKEMON_TYPE = """
MATCH (p:Pokemon {name: $pokemon}), (t:Type {name: $type})
CREATE (p)-[:IS_TYPE]->(t)
"""

INSERT_POKEMON_ABILITY = """
MATCH (p:Pokemon {name: $pokemon}), (a:Ability {name: $ability})
CREATE (p)-[:HAS_ABILITY]->(a)
"""

INSERT_MOVE = """
CREATE (m:Move {name: $name, category: $category, power: $power, pp: $pp, accuracy: $accuracy})
"""

INSERT_POKEMON_MOVES = """
MATCH (p:Pokemon {name: $pokemon}), (m:Move {name: $move})
CREATE (p)-[:KNOWS]->(m)
"""

INSERT_MOVES_TYPE = """
MATCH (m:Move {name: $move}), (t:Type {name: $type})
CREATE (m)-[:IS_TYPE]->(t)
"""

with open('abilities.json5') as abilities:
    abilities_data = json5.load(abilities)
    for ability in abilities_data:
        exec_query("neo4j://localhost" , "neo4j", "password", INSERT_ABILITY, {"name": ability, "description": abilities_data[ability]["shortDesc"]})

with open('pokedex.json') as pokedex:
    with open('learnsets.json') as learnset:
        with open('moves.json') as moves:    
            pokedex_data = json.load(pokedex)
            learnset_data = json.load(learnset)
            moves_data = json.load(moves)

            # Types 
            types =  set([type for pokemon in pokedex_data for type in pokedex_data[pokemon]["types"] if pokedex_data[pokemon]["num"] > 0])
            for type in types:
                exec_query("neo4j://localhost:7687", "neo4j", "password", INSERT_TYPE, {"name": type})
        
            # Pokemons
            for pokemon in pokedex_data:
                if pokedex_data[pokemon]["num"] > 0 and "-" not in pokedex_data[pokemon]["name"]:
                    print(pokedex_data[pokemon])
                    exec_query("neo4j://localhost:7687", "neo4j", "password", INSERT_POKEMON, createPokemon(pokedex_data[pokemon]))
                    
                    # Relacion pokemon con tipos
                    for type in pokedex_data[pokemon]["types"]:
                        exec_query("neo4j://localhost:7687", "neo4j", "password", INSERT_POKEMON_TYPE, {"pokemon": pokedex_data[pokemon]["name"], "type": type})
                    
                    # Relacion pokemon con habilidades
                    for ability in pokedex_data[pokemon]["abilities"].values():
                        exec_query("neo4j://localhost:7687", "neo4j", "password", INSERT_POKEMON_ABILITY, {"pokemon": pokedex_data[pokemon]["name"], "ability": ability.lower()})

                    # Take 2 random moves from the learnset and find them in the moves.json to dump them in the database
                    random_move = moves_data[random.choice(list(learnset_data[pokemon]["learnset"]))]
                    print(random_move)
                    
                     

In [58]:
query = """
MATCH (p:Pokemon)-[:HAS_ABILITY]->(a:Ability)
WHERE a.name = $ability
RETURN p, a
"""

records, summary, keys = exec_query("neo4j://localhost:7687", "neo4j", "password", query, {"ability": "Torrent"})
for record in records:
    print(record)

In [ ]:
query = """
MATCH (p1:Pokemon)-[:IS_TYPE]->(t1:Type)
MATCH (p1)-[:HAS_ABILITY]->(a1:Ability)
WITH p1, collect(t1) as types, collect(a1) as abilities
MATCH (p2:Pokemon)-[:IS_TYPE]->(t2:Type)
MATCH (p2)-[:HAS_ABILITY]->(a2:Ability)
WHERE p1 <> p2 AND types = collect(t2) AND abilities = collect(a2)
RETURN p1, p2
"""

records, summary, keys = exec_query("neo4j://localhost:7687", "neo4j", "password", query, {})

for record in records:
    print(record)

In [ ]:
query = """
MATCH (p:Pokemon)-[:IS_TYPE]->(t:Type)
WITH p, collect(t) as types
MATCH (p)-[:HAS_ABILITY]->(a:Ability)
WITH p, types, collect(a) as abilities
RETURN p, types, abilities
"""

records, summary, keys = exec_query("neo4j://localhost:7687", "neo4j", "password", query, {})
for record in records:
    print(record)

QUERY_POKEMON_BY_HEIGHT = """
MATCH (p:Pokemon)
WHERE p.heightm >= $min_height AND p.heightm <= $max_height
RETURN p
"""

QUERY_POKEMON_BY_TYPE = """
MATCH (p:Pokemon)-[:IS_OF_TYPE]->(t:Type {name: $type})
RETURN p
"""

QUERY_POKEMON_BY_ABILITY = """
MATCH (p:Pokemon)-[:HAS_ABILITY]->(a:Ability {name: $ability})
return p
"""

QUERY_POKEMON_BY_MOVE = """
MATCH (p:Pokemon-[HAS_MOVE]->(m:Move {name: $move}))
return p
"""

QUERY_POKEMON_BY_NAME = """
MATCH (p:Pokemon)
WHERE p.name == $name
RETURN p
"""